In [40]:
#pakacges

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import glob
import os
import random
from pathlib import Path
from PIL import Image

In [41]:
train = "/fox project/train/"
test = "/fox project/test/"

train, test

('/fox project/train/', '/fox project/test/')

In [42]:
for dirpath, dirnames, filenames in os.walk("/fox project/"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 1 images in '/fox project/'.
There are 2 directories and 0 images in '/fox project/test'.
There are 0 directories and 445 images in '/fox project/test\fox'.
There are 0 directories and 120 images in '/fox project/test\no_fox'.
There are 2 directories and 0 images in '/fox project/train'.
There are 0 directories and 1568 images in '/fox project/train\fox'.
There are 0 directories and 1333 images in '/fox project/train\no_fox'.


In [43]:
#image sizes

image_hight = 128
image_wildth = 128

image_size = (image_hight,image_wildth)

In [44]:
#switching to sensor + resizing
image_transform = transforms.Compose([
transforms.Resize(size=image_size),
transforms.RandomHorizontalFlip(p=0.5),
transforms.ToTensor()
]) 

In [45]:
test_transform = transforms.Compose([
transforms.Resize(size=image_size),
transforms.ToTensor()
]) 

In [46]:
train_data = datasets.ImageFolder(train,image_transform,target_transform=None)
test_data = datasets.ImageFolder(test,test_transform,target_transform=None)

In [47]:
#classes investigation
print(train_data.classes)
len(train_data.classes)

['fox', 'no_fox']


2

In [48]:
workers = os.cpu_count()

train_load = DataLoader(dataset=train_data,batch_size=32,num_workers=workers,shuffle=True)
test_load = DataLoader(dataset=test_data,batch_size=32,num_workers=workers,shuffle=False)

In [49]:
train2_load = DataLoader(dataset=train_data,batch_size=32,num_workers=workers,shuffle=False)

In [50]:
len(train_load)

91

In [51]:
# Creating a CNN-based image classifier.
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
          nn.Conv2d(3, 64, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(64),
          nn.MaxPool2d(2))
        self.conv_layer_2 = nn.Sequential(
          nn.Conv2d(64, 512, 3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512),
          nn.MaxPool2d(2))
        self.conv_layer_3 = nn.Sequential(
          nn.Conv2d(512, 512, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512),
          nn.MaxPool2d(2)) 
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=2048, out_features=2))
    def forward(self, x: torch.Tensor):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.classifier(x)
        return x
# Instantiate an object.
model = ImageClassifier()

In [52]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [53]:
img_batch, label_batch = next(iter(train_load))

In [54]:
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
label_single

tensor(0)

In [55]:
model.eval()
with torch.inference_mode():
    pred = model(img_single)

print(torch.softmax(pred, dim=1))

tensor([[0.5043, 0.4957]])


In [56]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_load, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / len(train_load):.3f}')
            running_loss = 0.0

print('Finished Training')


[1,    10] loss: 0.076
[1,    20] loss: 0.076
[1,    30] loss: 0.076
[1,    40] loss: 0.076
[1,    50] loss: 0.076
[1,    60] loss: 0.076
[1,    70] loss: 0.076
[1,    80] loss: 0.076
[1,    90] loss: 0.076
[2,    10] loss: 0.076
[2,    20] loss: 0.076
[2,    30] loss: 0.076
[2,    40] loss: 0.076
[2,    50] loss: 0.075
[2,    60] loss: 0.076
[2,    70] loss: 0.075
[2,    80] loss: 0.075
[2,    90] loss: 0.076
[3,    10] loss: 0.076
[3,    20] loss: 0.076
[3,    30] loss: 0.076
[3,    40] loss: 0.076
[3,    50] loss: 0.075
[3,    60] loss: 0.075
[3,    70] loss: 0.076
[3,    80] loss: 0.075
[3,    90] loss: 0.076
[4,    10] loss: 0.076
[4,    20] loss: 0.076
[4,    30] loss: 0.076
[4,    40] loss: 0.075
[4,    50] loss: 0.075
[4,    60] loss: 0.076
[4,    70] loss: 0.075
[4,    80] loss: 0.076
[4,    90] loss: 0.075
[5,    10] loss: 0.076
[5,    20] loss: 0.075
[5,    30] loss: 0.075
[5,    40] loss: 0.075
[5,    50] loss: 0.075
[5,    60] loss: 0.075
[5,    70] loss: 0.075
[5,    80] 

In [57]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_load:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 78 %


In [83]:
custom_path = "D:/fox project/customfox.jpg"

In [84]:
low_res_pic = Image.open(custom_path)


In [85]:
low_res_test = test_transform(low_res_pic)

In [86]:
model.eval()

with torch.inference_mode():
    # Add an extra dimension to image
    custom_image_transformed_with_batch_size = low_res_test.unsqueeze(dim=0)
    
    # Print out different shapes
    print(f"Custom image transformed shape: {low_res_test.shape}")
    print(f"Unsqueezed custom image shape: {custom_image_transformed_with_batch_size.shape}")
    
    # Make a prediction on image with an extra dimension
    custom_image_pred = model(low_res_test.unsqueeze(dim=0))

Custom image transformed shape: torch.Size([3, 128, 128])
Unsqueezed custom image shape: torch.Size([1, 3, 128, 128])


In [88]:


# Let's convert them from logits -> prediction probabilities -> prediction labels
# Print out prediction logits
print(f"Prediction logits: {custom_image_pred}")

# Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
custom_image_pred_probs = torch.softmax(custom_image_pred, dim=1)
print(f"Prediction probabilities: {custom_image_pred_probs}")

# Convert prediction probabilities -> prediction labels
custom_image_pred_label = torch.argmax(custom_image_pred_probs, dim=1)
print(f"Prediction label: {custom_image_pred_label}")



Prediction logits: tensor([[ 0.0387, -0.0628]])
Prediction probabilities: tensor([[0.5254, 0.4746]])
Prediction label: tensor([0])
